In [123]:
import pandas as pd
import datetime

import warnings
warnings.filterwarnings('ignore')

In [246]:
def yesNoToBool(df):
    """
    Convert string columns with "Yes" or "No" values to boolean.

    Parameters
    ----------
    df : DataFrame
        The DataFrame to parse (in our case the DataWorld dataframe)

    Returns
    -------
    None
        Updates the dataframe in place.

    """
    
    for col in df.columns:
        col_values = sorted([x.lower() for x in df[col].unique() if type(x) == str])
        if col_values == ["no", "yes"]:
            df[col] = df[col].map({'Yes': True, 'No': False})

In [247]:
def combineRidesAndName(rideFiles, rideNames, dataWorldDf):
    """
    Combine ride files with data.world data

    Parameters
    ----------
    rideFiles : list
        list of csv files with wait times to parse
        
    rideNames : list
        list of Ride names - must match order of rideFiles & data.world ride name
    

    Returns
    -------
    DataFrame
        Returns dataframe with all rides merged with their respective data.world metadata

    """
    
    all_rides_with_dw_metadata = []

    for idx, ride in enumerate(rideFiles):
        rideWaits = pd.read_csv(ride)
        rideWaits["Ride_name"] = rideNames[idx]
        
        
        ride_waits_with_metadata = rideWaits.merge(dataWorldDf, how="left")
        all_rides_with_dw_metadata.append(ride_waits_with_metadata)
    
    return pd.concat(all_rides_with_dw_metadata, ignore_index=True)   

In [248]:
def dateCleaning(df):
    """
    Clean & update date columns:
        datetime - converted to DateTime datatype
        date - converted to DateTime datatype
        Age_of_ride_days - updated to be the age of ride in days on that exact date in the park 
        Age_of_ride_years - updated to be the age of ride in years on that exact date in the park (Age_of_ride_days/365)
        Age_of_ride_total  - dropped because not useful for model
    

    Parameters
    ----------
    df : DataFrame
        dataframe with data.world columns and ride times
        
    Returns
    -------
    None
        updates df in place

    """
    
    df['datetime'] = pd.to_datetime(df["datetime"])
    df['date'] = pd.to_datetime(df["date"])
    
    df["Age_of_ride_days"] = (df["date"]-df["Open_date"]).dt.days
    df["Age_of_ride_years"] = df["Age_of_ride_days"]/365
    
    df.drop("Age_of_ride_total", axis=1, inplace=True)
    

In [249]:
def cleanData(rideFiles, rideNames):
    
    # load data.world metadata and filter for Magic Kingdom
    data_world = pd.read_excel("WDW_Ride_Data_DW.xlsx")
    mk_dw = data_world[data_world["Park_location"]=="MK"]
        
    yesNoToBool(mk_dw) # convert Yes/No columns to boolean
    
    all_rides_with_dw_metadata = combineRidesAndName(rideFiles, rideNames, mk_dw) #combine wait time data with data.world metadata
    
    dateCleaning(all_rides_with_dw_metadata) # clean date columns

    
    return all_rides_with_dw_metadata

In [250]:
rideTest = ["7_dwarfs_train.csv", "astro_orbiter.csv", "barnstormer.csv"]
rideNamesTest = ["Seven Dwarfs Mine Train", "Astro Orbiter", "The Barnstormer"]

allRides = cleanData(rideTest, rideNamesTest)

allRides

,date,datetime,SACTMIN,SPOSTMIN,Ride_name,Park_location,Park_area,Ride_type_all,Ride_type_thrill,Ride_type_spinning,...,Age_interest_tweens,Age_interest_teens,Age_interest_adults,Height_req_inches,Ride_duration_min,Open_date,Age_of_ride_days,Age_of_ride_years,TL_rank,TA_Stars
0,2015-01-01,2015-01-01 07:51:12,NaN,45.0,Seven Dwarfs Mine Train,MK,Fantasyland,"thrill, small drops",True,False,...,True,True,True,38,2.5,2014-05-28,218,0.597260,10.0,4.5
1,2015-01-01,2015-01-01 08:02:13,NaN,60.0,Seven Dwarfs Mine Train,MK,Fantasyland,"thrill, small drops",True,False,...,True,True,True,38,2.5,2014-05-28,218,0.597260,10.0,4.5
2,2015-01-01,2015-01-01 08:05:30,54.0,NaN,Seven Dwarfs Mine Train,MK,Fantasyland,"thrill, small drops",True,False,...,True,True,True,38,2.5,2014-05-28,218,0.597260,10.0,4.5
3,2015-01-01,2015-01-01 08:09:12,NaN,60.0,Seven Dwarfs Mine Train,MK,Fantasyland,"thrill, small drops",True,False,...,True,True,True,38,2.5,2014-05-28,218,0.597260,10.0,4.5
4,2015-01-01,2015-01-01 08:16:12,NaN,60.0,Seven Dwarfs Mine Train,MK,Fantasyland,"thrill, small drops",True,False,...,True,True,True,38,2.5,2014-05-28,218,0.597260,10.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923228,2021-12-28,2021-12-28 22:35:29,NaN,5.0,The Barnstormer,MK,Fantasyland,small drops,False,False,...,False,False,False,35,1.0,1996-10-01,9219,25.257534,36.0,3.5
923229,2021-12-28,2021-12-28 22:42:15,NaN,5.0,The Barnstormer,MK,Fantasyland,small drops,False,False,...,False,False,False,35,1.0,1996-10-01,9219,25.257534,36.0,3.5
923230,2021-12-28,2021-12-28 22:48:12,NaN,5.0,The Barnstormer,MK,Fantasyland,small drops,False,False,...,False,False,False,35,1.0,1996-10-01,9219,25.257534,36.0,3.5
923231,2021-12-28,2021-12-28 22:54:10,NaN,5.0,The Barnstormer,MK,Fantasyland,small drops,False,False,...,False,False,False,35,1.0,1996-10-01,9219,25.257534,36.0,3.5


In [251]:
allRides.iloc[0]

date                                 2015-01-01 00:00:00
datetime                             2015-01-01 07:51:12
SACTMIN                                              NaN
SPOSTMIN                                            45.0
Ride_name                        Seven Dwarfs Mine Train
Park_location                                         MK
Park_area                                    Fantasyland
Ride_type_all                        thrill, small drops
Ride_type_thrill                                    True
Ride_type_spinning                                 False
Ride_type_slow                                     False
Ride_type_small_drops                               True
Ride_type_big_drops                                False
Ride_type_dark                                     False
Ride_type_scary                                       No
Ride_type_water                                    False
Fast_pass                                           True
Classic                        

In [252]:
allRides["Ride_name"].unique()

array(['Seven Dwarfs Mine Train', 'Astro Orbiter', 'The Barnstormer'],
      dtype=object)